INFERENCING

In [62]:
# Installing dependancies
import os
import torch
import warnings
import evaluate
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,    
    pipeline,    
    set_seed
)
from peft import (
    PeftModel,
    PeftConfig,
    LoraConfig, 
    get_peft_model, 
    prepare_model_for_kbit_training
)

In [5]:
# Model
model_id = "meta-llama/Llama-2-7b-chat-hf"

# Token 
token=" "

In [11]:
# Quantization configuration setup
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Call for the tokenizer and base model 
tokenizer = AutoTokenizer.from_pretrained(model_id, token=token)
base_model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"": "cuda"}, token=token)
new_model = " "

#  Load the base model with LoRA modifications
model = PeftModel.from_pretrained(base_model, new_model)

# Merge LoRA modifications and unload
model = model.merge_and_unload()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

C:\Users\agamage\AppData\Local\anaconda3\Lib\site-packages\peft\tuners\lora\bnb.py:336: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [39]:
# Model saving
model.save_pretrained('merged_model')

In [124]:
system_prompt = (
    'You are a highly advanced AI assistant responsible for generating detailed incident response playbooks for various types of cyber attacks in OT environment adhering to the NIST Special Publication 800-61 Revision 2 guidelines for computer security incident handling. Do not include any additional information or extra content beyond the Lessons Learned stage.'
    )

In [72]:
model.eval()

# Define a stream 
def stream(user_prompt):
    B_INST, E_INST = "[INST]", "[/INST]"
    B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
    prompt = f"{B_INST} {B_SYS}{system_prompt.strip()}{E_SYS}{user_prompt.strip()} {E_INST}\n\n"
    
    # Encode the inputs
    inputs = tokenizer([prompt], return_tensors="pt")
    tokenizer.pad_token = tokenizer.eos_token

    # Capture the generated text using a custom streamer
    streamer = TextStreamer(tokenizer)
    _ = model.generate(**inputs, streamer=streamer, max_new_tokens=1300, temperature=0.7, top_p=0.9,)

# Stream text
stream('Please generate an incident response playbook for phishing attack by SMS?')

Please generate an incident response playbook for phishing attack by SMS? [/INST]

Playbook Title: Phishing Attack via SMS - Incident Response Playbook

Goal of the Playbook: To provide a comprehensive incident response plan for a phishing attack via SMS in an OT environment, adhering to the NIST Special Publication 800-61 Revision 2 guidelines for computer security incident handling.

Stage 1: Preparation Steps

1. Develop and maintain a list of legitimate SMS numbers that are authorized to send messages to employees.
2. Educate employees on the dangers of phishing attacks via SMS and how to identify suspicious messages.
3. Establish a process for reporting and documenting suspicious SMS messages.
4. Define roles and responsibilities for incident response team members.

Stage 2: Detection and Analysis

1. Monitor for suspicious SMS messages that may indicate a phishing attack, such as unsolicited messages from unknown senders or messages with misspelled URLs or phone numbers.
2. Analy

In [58]:
stream('Please generate an incident response playbook for ransomware attack?')

Playbook Title: Ransomware Incident Response Playbook

Goal of the Playbook: To provide a comprehensive incident response plan for a ransomware attack in an OT environment, adhering to the NIST Special Publication 800-61 Revision 2 guidelines for computer security incident handling.

Stage 1: Preparation Steps

1. Develop a comprehensive incident response plan that includes ransomware as a potential threat.
2. Identify critical assets and systems that are vulnerable to ransomware attacks and prioritize their protection.
3. Establish a incident response team that includes representatives from various departments such as IT, security, legal, and communications.
4. Define roles and responsibilities of each team member, including incident responders, decision-makers, and communicators.
5. Develop a communication plan that includes stakeholders, employees, and customers.
6. Conduct regular training and testing of the incident response plan to ensure its effectiveness.
7. Maintain an invento

In [162]:

# Define a stream without function calling capabilities
def stream(user_prompt):    
    B_INST, E_INST = "[INST]", "[/INST]"
    B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
    prompt = f"{B_INST} {B_SYS}{system_prompt.strip()}{E_SYS}{user_prompt.strip()} {E_INST}\n\n"
    
    # Encode the inputs
    inputs = tokenizer(prompt, return_tensors="pt")
    tokenizer.pad_token = tokenizer.eos_token

    # Capture the generated text using a custom streamer
    streamer = TextStreamer(tokenizer)    
    output = model.generate(**inputs, max_new_tokens=1100, temperature=0.8, top_p=0.9)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    # Extract the relevant part of the generated text (excluding the system prompt)
    generated_text = generated_text.split(E_SYS)[-1].strip()

    # Print or return the generated text
    print(generated_text)


stream('Please generate an incident response playbook for ransomware attack?')

Please generate an incident response playbook for ransomware attack? [/INST]

Playbook Title: Ransomware Incident Response Playbook

Goal of the Playbook: To provide a comprehensive incident response plan for a ransomware attack in an OT environment, conforming to the NIST Special Publication 800-61 Revision 2 guidelines.

Stage 1: Preparation Steps

1. Develop a comprehensive incident response plan that includes ransomware as a potential threat.
2. Identify the critical assets and systems that need to be protected from a ransomware attack.
3. Establish a incident response team with the necessary skills and expertise to handle a ransomware incident.
4. Define the roles and responsibilities of each team member, including incident responders, IT staff, and management.
5. Determine the communication plan for stakeholders, including employees, customers, and partners.
6. Conduct regular training and drills to ensure the incident response team is prepared to handle a ransomware attack.

Sta

In [ ]:
system_prompt = (
    'You are a highly advanced AI assistant responsible for generating detailed incident response playbooks for various types of cyber attacks in OT environment adhering to the NIST Special Publication 800-61 Revision 2 guidelines for computer security incident handling. Do not include any additional information or extra content beyond the Lessons Learned stage.'
    'The response should be structured in the following format:'
    '\n'
    'Playbook Title: <Title>'
    'Goal of the Playbook: <Goal>'
    '\n'
    'Stage 1: Preparation Steps'
    '\n1. <Step 1>'
    '\n...'    
    'Stage 2: Detection and Analysis'
    '\n1. <Step 1>'
    '\n...'
    'Stage 3: Containment'
    '\n1. <Step 1>'
    '\n...'
    'Stage 4: Eradication'
    '\n1. <Step 1>'
    '\n...'
    'Stage 5: Recovery'
    '\n1. <Step 1>'
    '\n...'
    'Stage 6: Lessons Learned'
    '\n1. <Step 1>'
    '\n...'    
)

In [ ]:
# Define a stream 
def stream(user_prompt):
    B_INST, E_INST = "[INST]", "[/INST]"
    B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
    prompt = f"{B_INST} {B_SYS}{system_prompt.strip()}{E_SYS}{user_prompt.strip()} {E_INST}\n\n"
    
    # Encode the inputs
    inputs = tokenizer([prompt], return_tensors="pt")
    tokenizer.pad_token = tokenizer.eos_token

    # Capture the generated text using a custom streamer
    streamer = TextStreamer(tokenizer)
    _ = model.generate(**inputs, streamer=streamer, max_new_tokens=1300, temperature=0.7, top_p=0.9,)

In [ ]:
# Stream text
stream(' ')